# File Reader

Copyright 2022 Michael George (AKA Logiqx).

This file is part of [GPS Wizard](https://logiqx.github.io/gps-wizard/) and is distributed under the terms of the GNU General Public License.

GPS Wizard is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

GPS Wizard is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with GPS Wizard. If not, see <https://www.gnu.org/licenses/>.

In [1]:
import sys
import os

import unittest

for path in ['python', '.', '..']:
    readersPath = os.path.join(path, 'readers')
    if readersPath not in sys.path:
        sys.path.extend([readersPath])

from fit_reader import FitReader
from gpx_reader import GpxReader
from nmea_reader import NmeaReader
from oao_reader import OaoReader
from sbn_reader import SbnReader
from sbp_reader import SbpReader
from ubx_reader import UbxReader

## Main Method

In [2]:
def getFileReader(filename):
    '''Determine file type and return appropriate file reader'''

    ext = os.path.splitext(filename)[1].lower()

    if ext == '.fit':
        fileReader = FitReader(filename)
    elif ext == '.gpx':
        fileReader = GpxReader(filename)
    elif ext in ('.nmea', '.txt'):
        fileReader = NmeaReader(filename)
    elif ext == '.oao':
        fileReader = OaoReader(filename)
    elif ext == '.sbn':
        fileReader = SbnReader(filename)
    elif ext == '.sbp':
        fileReader = SbpReader(filename)
    elif ext == '.ubx':
        fileReader = UbxReader(filename)
    else:
        raise RuntimeError('Extension {} is not supported'.format(ext))

    return fileReader

## Unit Tests

In [3]:
class TestReaders(unittest.TestCase):
    '''Class to test all of the file readers'''

    def testFitApexPro(self):
        '''Test loading of an FIT file from COROS Apex Pro'''

        filename = os.path.join(projdir, 'sessions', '20220411', 'APEX_Pro_Speedsurfing20220411111317.fit')
        fileReader = getFileReader(filename)
        fileReader.load()
        
        track = fileReader.tracks[0]

        self.assertEqual(track.fields['ts']['decimals'], 0)
        self.assertEqual(track.fields['lat']['decimals'], 7)
        self.assertEqual(track.fields['lon']['decimals'], 7)
        self.assertEqual(track.fields['dist']['decimals'], 1)
        self.assertEqual(track.fields['sog']['decimals'], 3)
        self.assertEqual(track.fields['cog']['decimals'], 0)
        self.assertEqual(track.fields['hdop']['decimals'], 1)


    def testGpxApexPro(self):
        '''Test loading of an GPX file from COROS Apex Pro'''

        filename = os.path.join(projdir, 'sessions', '20220411', 'APEX_Pro_Speedsurfing20220411111317.gpx')
        fileReader = getFileReader(filename)
        fileReader.load()

        track = fileReader.tracks[0]

        self.assertEqual(track.fields['ts']['decimals'], 0)
        self.assertEqual(track.fields['lat']['decimals'], 7)
        self.assertEqual(track.fields['lon']['decimals'], 7)
        self.assertEqual(track.fields['cog']['decimals'], 0)
        self.assertEqual(track.fields['hdop']['decimals'], 1)


    def testGpxSbn(self):
        '''Test loading of an GPX file from SBN file'''

        filename = os.path.join(projdir, 'sessions', '20220411', 'GT31_1Hz_GEORG30MICHA_932000175_20220411_111600.gpx')
        fileReader = getFileReader(filename)
        fileReader.load()

        track = fileReader.tracks[0]

        self.assertEqual(track.fields['ts']['decimals'], 0)
        self.assertEqual(track.fields['lat']['decimals'], 7)
        self.assertEqual(track.fields['lon']['decimals'], 7)
        self.assertEqual(track.fields['ele']['decimals'], 2)
        self.assertEqual(track.fields['cog']['decimals'], 2)
        self.assertEqual(track.fields['sog']['decimals'], 2)
        self.assertEqual(track.fields['hdop']['decimals'], 1)


    def testGpxSbp(self):
        '''Test loading of an GPX file from SBP file'''

        filename = os.path.join(projdir, 'sessions', '20220411', 'GT31_1Hz_GEORG30MICHA_932000175_20220512_094254_DLG.gpx')
        fileReader = getFileReader(filename)
        fileReader.load()

        track = fileReader.tracks[0]

        self.assertEqual(track.fields['ts']['decimals'], 0)
        self.assertEqual(track.fields['lat']['decimals'], 7)
        self.assertEqual(track.fields['lon']['decimals'], 7)
        self.assertEqual(track.fields['ele']['decimals'], 2)
        self.assertEqual(track.fields['cog']['decimals'], 2)
        self.assertEqual(track.fields['sog']['decimals'], 2)
        self.assertEqual(track.fields['hdop']['decimals'], 1)


    def testGpxFenix6(self):
        '''Test loading of an GPX file from Fenix 6'''

        filename = os.path.join(projdir, 'sessions', '20220422', 'activity_8686414511.gpx')
        fileReader = getFileReader(filename)
        fileReader.load()

        track = fileReader.tracks[0]

        self.assertEqual(track.fields['ts']['decimals'], 0)
        self.assertEqual(track.fields['lat']['decimals'], 7)
        self.assertEqual(track.fields['lon']['decimals'], 7)
        self.assertEqual(track.fields['ele']['decimals'], 1)


    def testGpxOsMaps(self):
        '''Test loading of an GPX file from OS Maps'''

        filename = os.path.join(projdir, 'sessions', 'misc', 'Seatownandgoldencap.gpx')
        fileReader = getFileReader(filename)
        fileReader.load()

        track = fileReader.tracks[0]

        self.assertEqual(track.fields['lat']['decimals'], 7)
        self.assertEqual(track.fields['lon']['decimals'], 7)


    def testNmeaGt31(self):
        '''Test loading of an NMEA file from a GT-31'''

        filename = os.path.join(projdir, 'sessions', '20220411', 'GT31_1Hz_GEORG30MICHA_932000175_20220411_111600.nmea')
        fileReader = getFileReader(filename)
        fileReader.load()

        track = fileReader.tracks[0]

        self.assertEqual(track.fields['ts']['decimals'], 0)
        self.assertEqual(track.fields['lat']['decimals'], 7)
        self.assertEqual(track.fields['lon']['decimals'], 7)
        self.assertEqual(track.fields['sog']['decimals'], 3)
        self.assertEqual(track.fields['cog']['decimals'], 2)
        self.assertEqual(track.fields['hdop']['decimals'], 1)
        self.assertEqual(track.fields['ele']['decimals'], 2)


    def testOaoMini(self):
        '''Test loading of an OAO file from a Motion Mini'''

        filename = os.path.join(projdir, 'sessions', '20220411', 'Motion_Mini_10Hz_0470_2022-04-11-1117.oao')
        fileReader = getFileReader(filename)
        fileReader.load()

        track = fileReader.tracks[0]

        self.assertEqual(track.fields['lat']['decimals'], 7)
        self.assertEqual(track.fields['lon']['decimals'], 7)
        self.assertEqual(track.fields['ele']['decimals'], 3)
        self.assertEqual(track.fields['sog']['decimals'], 3)
        self.assertEqual(track.fields['cog']['decimals'], 3)
        self.assertEqual(track.fields['ts']['decimals'], 1)
        self.assertEqual(track.fields['sacc']['decimals'], 3)
        self.assertEqual(track.fields['hacc']['decimals'], 3)
        self.assertEqual(track.fields['vacc']['decimals'], 3)
        self.assertEqual(track.fields['cacc']['decimals'], 3)
        self.assertEqual(track.fields['hdop']['decimals'], 2)


    def testOaoEsp(self):
        '''Test loading of an OAO file from a ESP-GPS'''

        filename = os.path.join(projdir, 'sessions', '20211230-esp', 'Head_L_7C9EBDFAF5C8_007.oao')
        fileReader = getFileReader(filename)
        fileReader.load()

        track = fileReader.tracks[0]

        self.assertEqual(track.fields['lat']['decimals'], 7)
        self.assertEqual(track.fields['lon']['decimals'], 7)
        self.assertEqual(track.fields['ele']['decimals'], 3)
        self.assertEqual(track.fields['sog']['decimals'], 3)
        self.assertEqual(track.fields['cog']['decimals'], 3)
        self.assertEqual(track.fields['ts']['decimals'], 3)
        self.assertEqual(track.fields['sacc']['decimals'], 3)
        self.assertEqual(track.fields['hacc']['decimals'], 3)
        self.assertEqual(track.fields['vacc']['decimals'], 3)
        self.assertEqual(track.fields['cacc']['decimals'], 3)
        self.assertEqual(track.fields['hdop']['decimals'], 2)


    def testSbnGt11(self):
        '''Test loading of an SBN file from a GT-11'''

        filename = os.path.join(projdir, 'sessions', '20071227', 'MIKE_G_1003053_20071227_133836.SBN')
        fileReader = getFileReader(filename)
        fileReader.load()

        track = fileReader.tracks[0]

        self.assertEqual(track.fields['ts']['decimals'], 3)
        self.assertEqual(track.fields['lat']['decimals'], 7)
        self.assertEqual(track.fields['lon']['decimals'], 7)
        self.assertEqual(track.fields['ele']['decimals'], 2)
        self.assertEqual(track.fields['sog']['decimals'], 2)
        self.assertEqual(track.fields['cog']['decimals'], 2)
        self.assertEqual(track.fields['roc']['decimals'], 2)
        self.assertEqual(track.fields['ehpe']['decimals'], 2)
        self.assertEqual(track.fields['evpe']['decimals'], 2)
        self.assertEqual(track.fields['hdop']['decimals'], 1)
        self.assertEqual(track.fields['sogu']['decimals'], 2)
        self.assertEqual(track.fields['cogu']['decimals'], 2)


    def testSbnGt31(self):
        '''Test loading of an SBN file from a GT-31'''

        filename = os.path.join(projdir, 'sessions', '20220411', 'GT31_1Hz_GEORG30MICHA_932000175_20220411_111600.SBN')
        fileReader = getFileReader(filename)
        fileReader.load()

        track = fileReader.tracks[0]

        self.assertEqual(track.fields['ts']['decimals'], 0)
        self.assertEqual(track.fields['lat']['decimals'], 7)
        self.assertEqual(track.fields['lon']['decimals'], 7)
        self.assertEqual(track.fields['ele']['decimals'], 2)
        self.assertEqual(track.fields['sog']['decimals'], 2)
        self.assertEqual(track.fields['cog']['decimals'], 2)
        self.assertEqual(track.fields['roc']['decimals'], 2)
        self.assertEqual(track.fields['ehpe']['decimals'], 2)
        self.assertEqual(track.fields['evpe']['decimals'], 2)
        self.assertEqual(track.fields['hdop']['decimals'], 1)
        self.assertEqual(track.fields['sogu']['decimals'], 2)
        self.assertEqual(track.fields['cogu']['decimals'], 2)
        self.assertEqual(track.fields['sdop']['decimals'], 2)
        self.assertEqual(track.fields['vsdop']['decimals'], 2)


    def testSbpGt11(self):
        '''Test loading of an SBP file from a GT-11'''

        filename = os.path.join(projdir, 'sessions', '20071227', 'MIKE_G_1003053_20071227_165512_DLG.SBP')
        fileReader = getFileReader(filename)
        fileReader.load()

        track = fileReader.tracks[0]

        self.assertEqual(track.fields['ts']['decimals'], 3)
        self.assertEqual(track.fields['lat']['decimals'], 7)
        self.assertEqual(track.fields['lon']['decimals'], 7)
        self.assertEqual(track.fields['ele']['decimals'], 2)
        self.assertEqual(track.fields['sog']['decimals'], 2)
        self.assertEqual(track.fields['cog']['decimals'], 2)
        self.assertEqual(track.fields['roc']['decimals'], 2)
        self.assertEqual(track.fields['hdop']['decimals'], 1)



    def testSbpGt31(self):
        '''Test loading of an SBP file from a GT-31'''

        filename = os.path.join(projdir, 'sessions', '20220411', 'GT31_1Hz_GEORG30MICHA_932000175_20220512_094254_DLG.SBP')
        fileReader = getFileReader(filename)
        fileReader.load()

        track = fileReader.tracks[0]

        self.assertEqual(track.fields['ts']['decimals'], 0)
        self.assertEqual(track.fields['lat']['decimals'], 7)
        self.assertEqual(track.fields['lon']['decimals'], 7)
        self.assertEqual(track.fields['ele']['decimals'], 2)
        self.assertEqual(track.fields['sog']['decimals'], 2)
        self.assertEqual(track.fields['cog']['decimals'], 2)
        self.assertEqual(track.fields['roc']['decimals'], 2)
        self.assertEqual(track.fields['hdop']['decimals'], 1)
        self.assertEqual(track.fields['sdop']['decimals'], 2)
        self.assertEqual(track.fields['vsdop']['decimals'], 2)


    def testUbxEsp(self):
        '''Test loading of a UBX file from an ESP-GPS'''

        filename = os.path.join(projdir, 'sessions', '20211230-esp', 'Head_L_7C9EBDFAF5C8_007.ubx')
        fileReader = getFileReader(filename)
        fileReader.load()
        
        track = fileReader.tracks[0]

        self.assertEqual(track.fields['ts']['decimals'], 3)
        self.assertEqual(track.fields['lat']['decimals'], 7)
        self.assertEqual(track.fields['lon']['decimals'], 7)
        self.assertEqual(track.fields['ele']['decimals'], 3)
        self.assertEqual(track.fields['hacc']['decimals'], 3)
        self.assertEqual(track.fields['vacc']['decimals'], 3)
        self.assertEqual(track.fields['roc']['decimals'], 3)
        self.assertEqual(track.fields['sog']['decimals'], 3)
        self.assertEqual(track.fields['cog']['decimals'], 3)
        self.assertEqual(track.fields['sacc']['decimals'], 3)
        self.assertEqual(track.fields['cacc']['decimals'], 3)
        self.assertEqual(track.fields['pdop']['decimals'], 2)

In [4]:
if __name__ == '__main__':
    # Determine whether session is interactive or batch to facilitate unittest.main(..., exit=testExit)
    import __main__ as main
    testExit = hasattr(main, '__file__')

    projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

    unittest.main(argv=['first-arg-is-ignored'], exit=testExit)

..............
----------------------------------------------------------------------
Ran 14 tests in 3.574s

OK
